In [1]:
import asyncio
import websockets
import json
import pandas as pd
import datetime as dt
import plotly.graph_objects as go
import datetime
import time
import nest_asyncio     #run this to overcome loop error in pandas notebook, in python its not needed 
nest_asyncio.apply()


In [2]:

#Get Unix Timestamp to use in __main__ function, only use the first 13 digits

start_date = datetime.datetime(2020, 7, 1)
end_date = datetime.datetime.now()



# Convert to Unix 13 digits 
unix_start_date = int(datetime.datetime.timestamp(start_date))*1000
print("Start Date Unix:"+str(unix_start_date))


unix_end_date = int(datetime.datetime.timestamp(end_date))*1000
print("End Date Unix:"+ str(unix_end_date))

Start Date Unix:1593554400000
End Date Unix:1665229982000


In [3]:
async def call_api(msg):
   async with websockets.connect('wss://test.deribit.com/ws/api/v2') as websocket:
       await websocket.send(msg)
       while websocket.open:
           response = await websocket.recv()
           return response



def async_loop(api, message):
    return asyncio.get_event_loop().run_until_complete(api(message))


def retrieve_historic_data(start, end, instrument, timeframe):
    msg = \
        {
            "jsonrpc": "2.0",
            "id": 833,
            "method": "public/get_tradingview_chart_data",
            "params": {
                "instrument_name": instrument,
                "start_timestamp": start,
                "end_timestamp": end,
                "resolution": timeframe
            }
        }
    resp = async_loop(call_api, json.dumps(msg))

    return resp

def json_to_dataframe(json_resp):
    res = json.loads(json_resp)

    df = pd.DataFrame(res['result'])

    df['ticks'] = df.ticks / 1000
    df['timestamp'] = [dt.datetime.fromtimestamp(date) for date in df.ticks]

    return df



if __name__ == '__main__':
    start = unix_start_date                 #Exchange this value with the printed value on cell 2 without .0 ex: 1554373800000
    end = unix_end_date                   #Exchange this value with the printed value on cell 2 without .0 ex: 1554376800000
    instrument = "BTC-PERPETUAL"
    timeframe = '60'                        #Time frames: 1, 3, 5, 10, 15, 30, 60, 120, 180, 360, 720, 1D

    json_resp = retrieve_historic_data(start, end, instrument, timeframe)

    df = json_to_dataframe(json_resp)
    #print(df.head())

df.head()

,volume,ticks,status,open,low,high,cost,close,timestamp
0,1509.042782,1.647227e+09,ok,38036.0,37446.0,38728.0,57376870.0,38162.5,2022-03-14 04:00:00
1,549.328522,1.647230e+09,ok,38163.0,38140.5,38944.0,21187340.0,38539.5,2022-03-14 05:00:00
2,150.289359,1.647234e+09,ok,38539.5,38470.0,38616.5,5790450.0,38505.5,2022-03-14 06:00:00
3,221.880542,1.647238e+09,ok,38480.0,38480.0,38802.5,8568700.0,38659.0,2022-03-14 07:00:00
4,3815.452296,1.647241e+09,ok,38652.0,37975.5,39576.5,148294420.0,39084.5,2022-03-14 08:00:00


In [4]:
# Remove unnecessary columns
df.drop(["volume", "ticks", "status", "cost" ], axis=1, inplace=True)
df.rename(columns = {'timestamp':'date'}, inplace = True)
df.set_index('date', inplace=True)

#Change columns order to use on the volatility spreadsheet
df=df[["open", "high", "low", "close"]]



#reset index if necessary 
#df.reset_index(inplace=True)

df

,open,high,low,close
date,,,,
2022-03-14 04:00:00,38036.0,38728.0,37446.0,38162.5
2022-03-14 05:00:00,38163.0,38944.0,38140.5,38539.5
2022-03-14 06:00:00,38539.5,38616.5,38470.0,38505.5
2022-03-14 07:00:00,38480.0,38802.5,38480.0,38659.0
2022-03-14 08:00:00,38652.0,39576.5,37975.5,39084.5
...,...,...,...,...
2022-10-08 09:00:00,19510.5,19523.5,19488.0,19500.0
2022-10-08 10:00:00,19500.5,19850.0,19445.0,19463.0
2022-10-08 11:00:00,19463.0,19506.5,19463.0,19490.0


In [5]:
#Save to csv 

#df.to_csv(r"/home/user/Documents/Python Programs /datasets/CryptoHistoricalData/BTC-USD Derebit 1m.csv", index = True )

In [6]:
#Plot the chart 

fig = go.Figure()
fig.update_layout(
    title={
        'text': f"{instrument} - {timeframe} min",
        'x':0.5,
        'xanchor': 'center'
    },
    xaxis_title="Date",
    yaxis_title="Price",
    xaxis_rangeslider_visible=False
)
fig.add_trace(
    go.Candlestick(
        x=df.index,
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close']
    )
)
fig.show()